# Training procedure:

Basic training pocedure includes making a word to index mapping based on the occurences of words present in dataset
, either go through the dataset and collect occurence of each word in a dictionary , and index them according to their occurences
.We can use Tokenizer class from keras library to collect word2index dictionary.

Then we make embedding matrix to be used by Embedding layer , we can either randomly initialize it or use already pretrained
embeddings on a large dataset such as word2vec ,or glove embeddings ,for this model glove embeddings has been used.See  <a href="https://blog.keras.io/using-pre-trained-word-embeddings-in-a-keras-model.html">this keras blog post</a> for more details.

We can get the multichannel.py from models/ directory , this will provide us with a model , we can initialise it as shown below

In [ ]:
f=open("data/pos_polarity.txt",encoding="utf8")
texts=[]
labels=[]
for line in f:
    texts.append(line)
    labels.append(1)
print("found {} texts".format(len(texts)) )

In [ ]:
f=open("data/neg_polarity.txt",encoding="utf8")
texts_2=[]
labels_2=[]
for line in f:
    texts_2.append(line)
    labels_2.append(0)
print("found {} texts".format(len(texts_2)) )

In [ ]:
texts=texts+texts_2
labels=labels+labels_2
len(texts),len(labels)

In [ ]:
sentence_len=[]
for sentence in texts:
    sentence_len.append(len(sentence))
plt.hist(sentence_len,cumulative=True,bins=40)

In [ ]:
vocab_size=15000
max_seq_len=250
embedding_dim=100
validation_split=0.7  


In [ ]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

tokenizer=Tokenizer(num_words=vocab_size)

tokenizer.fit_on_texts(texts)
sequences=tokenizer.texts_to_sequences(texts)
data=pad_sequences(sequences,maxlen=max_seq_len)

In [ ]:
#prepare the embedding layer ,using embedding index and
embedding_index={}
f=open("glove.6B.100d.txt",encoding="utf8")
for line in f:
    values=line.strip().split()
    word=values[0]
    
    if word  in tokenizer.word_index:
        embedding_index[word]=np.asarray(values[1:],dtype="float32")
        
print("embedding index ready")


embedding_matrix=np.zeros((vocab_size+1,embedding_dim))
 
for word,i in tokenizer.word_index.items():
    try:
        
        embedding_matrix[i]=embedding_index[word]
    except:
        
            try:
                embedding_matrix[i]=np.random.normal(scale=0.05,size=(embedding_dim,))
            except:
                print("index error")
                
                
print("Embedding matrix ready....")

There are mainly two models, providing drop argument will also add a dropout layer with p=0.5 after Global max pooling layer ,
another is multichannel spatial_dropout which add spatial dropout1d layers.
 it drops entire 1D feature maps instead of individual elements. If adjacent frames within feature maps are strongly correlated (as is normally the case in early convolution layers) then regular dropout will not regularize the activations and will otherwise just result in an effective learning rate decrease. In this case, SpatialDropout1D will help promote independence between feature maps and should be used instead.(Source : keras documentation)

In [ ]:
import models.multichannel
model=multichannel_spatial.multichannel_drop(nb_classes=1,vocab_size=vocab_size,matrix=embedding_matrix,input_len=max_seq_len,drop=False)
model.compile(optimizer="adam",loss="binary_crossentropy",metrics=["accuracy"])
model.load_weights("weights/polarity.h5")

In [ ]:
#prepare the data
indices=np.random.choice(data.shape[0],replace=False,size=data.shape[0])
labels=np.array(labels).reshape(-1,1)
num_validation_samples=int(validation_split*data.shape[0])
data=data[indices]
labels=labels[indices]
x_train,y_train=data[:num_validation_samples],labels[:num_validation_samples]
x_test,y_test=data[num_validation_samples:],labels[num_validation_samples:]
x_train.shape,x_test.shape

In [ ]:
model.fit(x_train,y_train,epochs=1,batch_size=128,validation_data=(x_test,y_test))

In [ ]:
pickle.dump(tokenizer.word_index,open("weights/word2index.p","wb"))
dic=pickle.load(open("weights/word2index.p","rb"))

In [ ]:
def preprocess(text):
    #preprocess any query to make it model compatible
    temp=[]
    for word in text.strip().split():
        print(word)
        if word in word2index: 
            if word2index[word] < vocab_size:
                temp.append(word2index[word])
            else:
                temp.append(vocab_size+1)
        
    print(temp)